In [ ]:
# Mengimpor pustaka yang esensial untuk manipulasi data,
# pemodelan, dan pra-pemrosesan.
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

In [ ]:
# --- Penyeimbangan Dataset ---
#
# Latar Belakang: Dataset Bot-IoT sangat tidak seimbang, dengan jumlah data
# serangan (attack=1) jauh lebih banyak daripada data normal (attack=0).
# Model yang dilatih pada data seperti ini akan cenderung bias terhadap kelas mayoritas.
#
# Tujuan: Membuat dataset yang lebih seimbang dengan melakukan undersampling
# pada kelas mayoritas (serangan).

# Membaca dua file dataset. Dtype untuk kolom port (5 dan 7) secara eksplisit
# diatur ke string untuk menghindari peringatan tipe data campuran.
df_1 = pd.read_csv('../data/data_1.csv', dtype={5: str, 7: str})
df_2 = pd.read_csv('../data/data_2.csv', dtype={5: str, 7: str})

# 1. Menggabungkan semua data normal (attack == 0) dari kedua file.
df_0_all = pd.concat([
    df_1[df_1['attack'] == 0],
    df_2[df_2['attack'] == 0]
])

# 2. Menggabungkan semua data serangan (attack == 1) dari kedua file.
df_1_all = pd.concat([
    df_1[df_1['attack'] == 1],
    df_2[df_2['attack'] == 1]
])

# 3. Melakukan undersampling: Mengambil sampel acak sebanyak 20.000 baris
# dari data serangan. `random_state` digunakan untuk memastikan hasil yang sama setiap kali.
df_1_sampled = df_1_all.sample(n=20000, random_state=42)

# 4. Menggabungkan kembali data normal dengan data serangan yang telah di-undersample.
df_balanced = pd.concat([df_0_all, df_1_sampled])

# 5. Mengacak urutan baris (shuffling) untuk memastikan data terdistribusi secara
# acak dan tidak berurutan berdasarkan kelas. Indeks di-reset.
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Memeriksa distribusi kelas akhir untuk memverifikasi hasil penyeimbangan.
print(df_balanced['attack'].value_counts())

attack
1    20000
0     6934
Name: count, dtype: int64


In [ ]:
# Menampilkan daftar nama kolom untuk inspeksi awal.
print(df_balanced.columns.tolist())

['pkSeqID', 'stime', 'flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'pkts', 'bytes', 'state', 'ltime', 'seq', 'dur', 'mean', 'stddev', 'smac', 'dmac', 'sum', 'min', 'max', 'soui', 'doui', 'sco', 'dco', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate', 'attack', 'category', 'subcategory ']


In [ ]:
# Membersihkan nama kolom dengan menghapus spasi di awal atau akhir.
# Langkah ini penting untuk mencegah kesalahan saat mengakses kolom berdasarkan nama.
df_balanced.columns = df_balanced.columns.str.strip()

In [ ]:
# --- Seleksi Fitur (Feature Selection) ---
#
# Berdasarkan penelitian terkait dataset Bot-IoT, beberapa fitur diidentifikasi
# tidak relevan atau berpotensi menyebabkan overfitting pada model.
# - pkSeqID, seq: Identifier unik yang tidak memiliki nilai prediktif.
# - saddr, daddr: Alamat IP spesifik dapat membuat model menghafal pola
#   serangan dari IP tertentu, bukan perilaku serangannya (overfitting).
# - stime, ltime: Timestamp absolut dihapus untuk membuat model lebih general.
#   Fitur berbasis waktu yang lebih relevan (seperti 'dur') tetap dipertahankan.
# - category, subcategory: Label yang terlalu spesifik. Dihapus karena fokus
#   pemodelan adalah deteksi biner (serangan atau bukan).
columns_to_drop = [
    'pkSeqID', 'saddr', 'daddr', 'ltime', 'stime', 'seq', 'category', 'subcategory'
]
df_balanced = df_balanced.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Menampilkan informasi ringkas tentang DataFrame setelah penghapusan kolom.
# Ini digunakan untuk memeriksa tipe data, jumlah kolom, dan nilai non-null.
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26934 entries, 0 to 26933
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   flgs    26934 non-null  object 
 1   proto   26934 non-null  object 
 2   sport   26534 non-null  object 
 3   dport   26534 non-null  object 
 4   pkts    26934 non-null  int64  
 5   bytes   26934 non-null  int64  
 6   state   26934 non-null  object 
 7   dur     26934 non-null  float64
 8   mean    26934 non-null  float64
 9   stddev  26934 non-null  float64
 10  smac    0 non-null      float64
 11  dmac    0 non-null      float64
 12  sum     26934 non-null  float64
 13  min     26934 non-null  float64
 14  max     26934 non-null  float64
 15  soui    0 non-null      float64
 16  doui    0 non-null      float64
 17  sco     0 non-null      float64
 18  dco     0 non-null      float64
 19  spkts   26934 non-null  int64  
 20  dpkts   26934 non-null  int64  
 21  sbytes  26934 non-null  int64  
 22

In [ ]:
# Menampilkan lima baris pertama dari DataFrame untuk verifikasi visual.
df_balanced.head()

,flgs,proto,sport,dport,pkts,bytes,state,dur,mean,stddev,...,sco,dco,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack
0,e,udp,52841,53,2,176,INT,5.047946,0.000000,0.0,...,NaN,NaN,2,0,176,0,0.198100,0.19810,0.00000,0
1,e,tcp,60134,6788,2,120,RST,0.000114,0.000114,0.0,...,NaN,NaN,1,1,60,60,8771.929688,0.00000,0.00000,1
2,e,tcp,56304,15224,2,120,RST,0.033741,0.033741,0.0,...,NaN,NaN,1,1,60,60,29.637531,0.00000,0.00000,1
3,e,tcp,59460,25280,2,120,RST,0.032499,0.032499,0.0,...,NaN,NaN,1,1,60,60,30.770178,0.00000,0.00000,1
4,e,tcp,54892,1883,6,662,RST,0.000244,0.000244,0.0,...,NaN,NaN,3,3,456,206,20491.802734,8196.72168,8196.72168,1


In [ ]:
# Menghapus kolom-kolom yang teridentifikasi hanya berisi nilai null (NaN).
# Kolom-kolom ini tidak memberikan informasi apa pun untuk proses pemodelan.
df_balanced.drop(columns=["smac", "dmac", "soui", "doui", "sco", "dco"], inplace=True)

In [ ]:
# Menampilkan kembali informasi DataFrame untuk memastikan kolom-kolom
# null telah berhasil dihapus.
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26934 entries, 0 to 26933
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   flgs    26934 non-null  object 
 1   proto   26934 non-null  object 
 2   sport   26534 non-null  object 
 3   dport   26534 non-null  object 
 4   pkts    26934 non-null  int64  
 5   bytes   26934 non-null  int64  
 6   state   26934 non-null  object 
 7   dur     26934 non-null  float64
 8   mean    26934 non-null  float64
 9   stddev  26934 non-null  float64
 10  sum     26934 non-null  float64
 11  min     26934 non-null  float64
 12  max     26934 non-null  float64
 13  spkts   26934 non-null  int64  
 14  dpkts   26934 non-null  int64  
 15  sbytes  26934 non-null  int64  
 16  dbytes  26934 non-null  int64  
 17  rate    26934 non-null  float64
 18  srate   26934 non-null  float64
 19  drate   26934 non-null  float64
 20  attack  26934 non-null  int64  
dtypes: float64(9), int64(7), object(5)


In [ ]:
# Menampilkan lima baris teratas setelah penghapusan kolom null.
df_balanced.head()

,flgs,proto,sport,dport,pkts,bytes,state,dur,mean,stddev,...,min,max,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack
0,e,udp,52841,53,2,176,INT,5.047946,0.000000,0.0,...,0.000000,0.000000,2,0,176,0,0.198100,0.19810,0.00000,0
1,e,tcp,60134,6788,2,120,RST,0.000114,0.000114,0.0,...,0.000114,0.000114,1,1,60,60,8771.929688,0.00000,0.00000,1
2,e,tcp,56304,15224,2,120,RST,0.033741,0.033741,0.0,...,0.033741,0.033741,1,1,60,60,29.637531,0.00000,0.00000,1
3,e,tcp,59460,25280,2,120,RST,0.032499,0.032499,0.0,...,0.032499,0.032499,1,1,60,60,30.770178,0.00000,0.00000,1
4,e,tcp,54892,1883,6,662,RST,0.000244,0.000244,0.0,...,0.000244,0.000244,3,3,456,206,20491.802734,8196.72168,8196.72168,1


In [ ]:
# Menghapus baris-baris duplikat untuk memastikan setiap entri data unik.
df_balanced.drop_duplicates(inplace=True)

In [ ]:
# Mengidentifikasi dan menghitung nilai yang hilang (missing values)
# pada setiap kolom.
missing = df_balanced.isnull().sum()
print(missing[missing > 0])

sport    307
dport    307
dtype: int64


In [ ]:
# Menangani nilai yang hilang dengan menghapus baris yang mengandungnya.
# Karena jumlah baris yang hilang relatif kecil, metode penghapusan (listwise deletion)
# dapat diterima tanpa menyebabkan kehilangan informasi yang signifikan.
df_balanced.dropna(subset=["sport", "dport"], inplace=True)

In [ ]:
# Verifikasi akhir untuk memastikan tidak ada lagi nilai yang hilang
# dalam dataset sebelum melanjutkan ke tahap pra-pemrosesan selanjutnya.
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26265 entries, 0 to 26933
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   flgs    26265 non-null  object 
 1   proto   26265 non-null  object 
 2   sport   26265 non-null  object 
 3   dport   26265 non-null  object 
 4   pkts    26265 non-null  int64  
 5   bytes   26265 non-null  int64  
 6   state   26265 non-null  object 
 7   dur     26265 non-null  float64
 8   mean    26265 non-null  float64
 9   stddev  26265 non-null  float64
 10  sum     26265 non-null  float64
 11  min     26265 non-null  float64
 12  max     26265 non-null  float64
 13  spkts   26265 non-null  int64  
 14  dpkts   26265 non-null  int64  
 15  sbytes  26265 non-null  int64  
 16  dbytes  26265 non-null  int64  
 17  rate    26265 non-null  float64
 18  srate   26265 non-null  float64
 19  drate   26265 non-null  float64
 20  attack  26265 non-null  int64  
dtypes: float64(9), int64(7), object(5)
memor

In [ ]:
# Memisahkan dataset menjadi matriks fitur (X) dan vektor target (y).
# 'attack' adalah variabel dependen yang akan diprediksi oleh model.
target = "attack"
X = df_balanced.drop(columns=[target])
y = df_balanced[target]

In [ ]:
# Memeriksa kembali distribusi kelas pada variabel target setelah
# semua proses pembersihan data selesai.
print(y.value_counts())

attack
1    19941
0     6324
Name: count, dtype: int64


In [ ]:
# --- Pra-pemrosesan: Encoding Fitur Kategorikal (Kardinalitas Tinggi) ---
#
# Menerapkan Label Encoding pada fitur 'sport' dan 'dport'.
# Label Encoding mengubah setiap nilai unik kategorikal menjadi sebuah integer unik.
# Metode ini dipilih untuk fitur dengan kardinalitas tinggi (banyak nilai unik)
# untuk menghindari pembuatan dimensi yang terlalu besar (seperti pada One-Hot Encoding).
high_card_cat = ["sport", "dport"]
for col in high_card_cat:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

In [ ]:
# Menampilkan distribusi nilai untuk kolom 'sport' setelah di-encode.
print(X['sport'].value_counts())

sport
8004    705
7444    693
8208    685
2375    683
8241    676
       ... 
3017      1
474       1
774       1
7541      1
6912      1
Name: count, Length: 9942, dtype: int64


In [ ]:
# Menampilkan distribusi nilai untuk kolom 'dport' setelah di-encode.
print(X['dport'].value_counts())

dport
8976     5125
11826    1726
11827     631
2395      293
8977      236
         ... 
2631        1
10925       1
4753        1
5612        1
5449        1
Name: count, Length: 12477, dtype: int64


In [ ]:
# --- Pra-pemrosesan: Encoding Fitur Kategorikal (Kardinalitas Rendah) ---
#
# Menerapkan One-Hot Encoding pada fitur dengan jumlah kategori unik yang sedikit.
# One-Hot Encoding membuat kolom biner baru untuk setiap kategori. Ini adalah
# metode yang lebih disukai untuk data kategorikal nominal (tanpa urutan)
# karena tidak menciptakan hubungan ordinal artifisial antar kategori.
low_card_cat = ["proto", "state", "flgs"]

ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
ohe_result = ohe.fit_transform(X[low_card_cat])
ohe_cols = ohe.get_feature_names_out(low_card_cat)

X_ohe = pd.DataFrame(ohe_result, columns=ohe_cols, index=X.index)

In [ ]:
# --- Pra-pemrosesan: Penskalaan Fitur Numerik ---
#
# Menerapkan StandardScaler pada semua fitur numerik.
# Penskalaan ini mengubah distribusi fitur sehingga memiliki rata-rata (mean) 0
# dan standar deviasi 1. Ini adalah langkah penting untuk banyak algoritma
# machine learning (seperti SVM, Logistic Regression) yang sensitif terhadap
# skala fitur, memastikan semua fitur memiliki kontribusi yang setara.
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Kolom yang sudah di-LabelEncode dikecualikan dari penskalaan.
num_cols = [col for col in num_cols if col not in high_card_cat]

scaler = StandardScaler()
scaled_result = scaler.fit_transform(X[num_cols])

X_scaled = pd.DataFrame(scaled_result, columns=num_cols, index=X.index)

In [ ]:
# Menggabungkan semua set fitur yang telah diproses (scaled, label-encoded,
# dan one-hot-encoded) menjadi satu matriks fitur akhir.
X_processed = pd.concat([X_scaled, X[high_card_cat], X_ohe], axis=1)

In [ ]:
# Membuat DataFrame final dengan menggabungkan matriks fitur yang
# telah diproses (X_processed) dengan vektor target (y).
final_df = pd.concat([X_processed, y], axis=1)

In [ ]:
# Menampilkan informasi ringkas dari DataFrame final untuk verifikasi akhir,
# termasuk jumlah total kolom dan tipe datanya.
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26265 entries, 0 to 26933
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pkts             26265 non-null  float64
 1   bytes            26265 non-null  float64
 2   dur              26265 non-null  float64
 3   mean             26265 non-null  float64
 4   stddev           26265 non-null  float64
 5   sum              26265 non-null  float64
 6   min              26265 non-null  float64
 7   max              26265 non-null  float64
 8   spkts            26265 non-null  float64
 9   dpkts            26265 non-null  float64
 10  sbytes           26265 non-null  float64
 11  dbytes           26265 non-null  float64
 12  rate             26265 non-null  float64
 13  srate            26265 non-null  float64
 14  drate            26265 non-null  float64
 15  sport            26265 non-null  int64  
 16  dport            26265 non-null  int64  
 17  proto_icmp       

In [ ]:
# Menampilkan lima baris pertama dari DataFrame yang sudah sepenuhnya diproses
# untuk memastikan semua transformasi telah diterapkan dengan benar.
final_df.head()

,pkts,bytes,dur,mean,stddev,sum,min,max,spkts,dpkts,...,flgs_e F,flgs_e t,flgs_e &,flgs_e *,flgs_e d,flgs_e g,flgs_e r,flgs_e s,flgs_eU,attack
0,-0.077967,-0.071110,-0.111816,-0.183322,-0.106786,-0.105419,-0.169021,-0.191095,-0.080777,-0.051129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,-0.077967,-0.071119,-0.135191,-0.183126,-0.106786,-0.105418,-0.168765,-0.190916,-0.081002,-0.050742,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,-0.077967,-0.071119,-0.135035,-0.125480,-0.106786,-0.105234,-0.093207,-0.138104,-0.081002,-0.050742,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,-0.077967,-0.071119,-0.135041,-0.127609,-0.106786,-0.105241,-0.095998,-0.140055,-0.081002,-0.050742,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,-0.077333,-0.071029,-0.135190,-0.182903,-0.106786,-0.105418,-0.168473,-0.190712,-0.080552,-0.049967,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
# Menyimpan DataFrame yang telah bersih dan terproses ke dalam file CSV baru.
# Ini memungkinkan penggunaan kembali data tanpa perlu mengulang
# semua langkah pra-pemrosesan.
final_df.to_csv("../Data/Bot_IoT_processedV2.csv", index=False)

In [ ]:
# Mencetak pesan konfirmasi dan perbandingan bentuk DataFrame
# sebelum dan sesudah pra-pemrosesan fitur kategorikal.
print("Preprocessing selesai. File disimpan sebagai Bot_IoT_processedV2.csv")
print("Shape sebelum:", df_balanced.shape)
print("Shape sesudah:", final_df.shape)

Preprocessing selesai. File disimpan sebagai Bot_IoT_processedV2.csv
Shape sebelum: (26265, 21)
Shape sesudah: (26265, 41)


In [ ]:
# Mengimpor pustaka joblib untuk menyimpan objek Python.
import joblib

# Menyimpan objek 'scaler' yang telah dilatih ke dalam sebuah file.
# Ini sangat penting agar transformasi penskalaan yang sama persis
# dapat diterapkan pada data baru (misalnya, data uji atau data produksi)
# untuk menjaga konsistensi.
joblib.dump(scaler, "../Data/scalerV2.pkl")

['../Data/scalerV2.pkl']